# 1. data preprocessing 및 dataset 구축하기

In [2]:
import numpy as np
#import cv2
# show error cases
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from keras.utils import np_utils

num_classes = 2
dataset_dir = "../dataset"
labelName_list = ["0", "1"]
channelName_list = ["BLUE", "GREEN", "IRR", "IRT", "RED"]
headingName_list = ["A", "B", "C", "D"]
flagToType_Dic = {"A" : [0, 1], "B" : [0, 1], "C" : [1, 0], "D" : [1, 0]}
typeName_list = ["FRONT", "BACK"]

def shuffle_data(data, label):
    idx = np.arange(len(data))
    np.random.shuffle(idx)
    
    shuffled_data = np.array(data)[np.array(idx)]
    shuffled_label = np.array(label)[np.array(idx)]
    
    return shuffled_data, shuffled_label, idx

def shuffle_data_in_defined(data, label, idx):
    shuffled_data = np.array(data)[np.array(idx)]
    shuffled_label = np.array(label)[np.array(idx)]
    
    return shuffled_data, shuffled_label, idx
    
def load_data(parent_dir, ext):
    
    image_Front_list = []
    image_Back_list = []
    label_Front_list = []
    label_Back_list = []
    fileFrontPath_list = []
    fileBackPath_list = []
    
    image_Dic = {'FRONT' : image_Front_list, 'BACK' : image_Back_list}
    label_Dic = {'FRONT' : label_Front_list, 'BACK' : label_Back_list}
    filePath_Dic = {'FRONT' : fileFrontPath_list, 'BACK' : fileBackPath_list}

    
    labelNames = os.listdir(parent_dir)
    for labelName in labelNames: # 하위 디렉토리 탐색      (label 폴더 : 0, 1)
        if not(any(labelName in tmpstr for tmpstr in labelName_list)):
            print("head ERR!!!")
            continue  # 해당 폴더가 투입면 폴더가 아닌 경우 다시 검색
        labelDirPath = os.path.join(parent_dir, labelName)
        print(labelName)
        if os.path.isdir(labelDirPath): # 디렉토리인 경우
            headingNames = os.listdir(labelDirPath)
            for headingName in headingNames: # 하위 디렉토리 탐색      (heading 폴더 : A, B, C, D)
                if not(any(headingName in tmpstr for tmpstr in headingName_list)):
                    print("head ERR!!!")
                    continue  # 해당 폴더가 투입면 폴더가 아닌 경우 다시 검색
                print('_' + headingName)
                headingDirPath = os.path.join(labelDirPath, headingName)

                if os.path.isdir(headingDirPath): # 디렉토리인 경우
                    channelNames = os.listdir(headingDirPath)
                    
                    #로드 리스트 생성
                    loadingImg_list = []
                    loadinglable_list = []
                    loadingName_list = []
                    
                    for channelName in channelNames:  # 하위 디렉토리 탐색      (각 Channel 접근 : R, G, B, IRR, IRT)
                        if not(any(channelName in tmpstr for tmpstr in channelName_list)):
                            print("channel ERR!!!")
                            continue  # 해당 폴더가 채널폴더가 아닌 경우 다시 검색
                        print('__' + channelName)
                        fileDirPath = os.path.join(headingDirPath, channelName)
                        
                        if os.path.isdir(fileDirPath): # 디렉토리인 경우
                            fileNames = os.listdir(fileDirPath)
                            for fileName in fileNames: #해당폴더 전체 이미지 탐색
                                # 순서대로 새로운 채널에 입력
                                filePath = os.path.join(fileDirPath, fileName)

                                _ext = os.path.splitext(filePath)[-1] # 파일 확장자 확인
                                if _ext == ext:
                                    print('.', end = '')
                                    image = mpimg.imread(filePath)
                                    if image is None:
                                        continue
                                
                                #이미지 크기 정규화
                                image = image.astype(np.float32) / 255.
                                #image = image.reshape(65, 308)
                                
                                #있는지 확인
                                if any(fileName in tmpstr for tmpstr in loadingName_list): #파일이 있는지 없는지 확인
                                    #있으면 dstack
                                    tmpidx = loadingName_list.index(fileName)
                                    loadingImg_list[tmpidx] = np.dstack((loadingImg_list[tmpidx], image))
                                    shapetmp = loadingImg_list[tmpidx].shape
                                    
                                    if channelName == "IRT":
                                        nameListLen = int(len(loadingName_list)/2 -1)
                                        #print(type(nameListLen))
                                        #print(loadingName_list)
                                        for chatchName in loadingName_list[nameListLen:] :
                                            if fileName[5:] in chatchName:
                                                findedName = chatchName
                                        tmp = loadingName_list.index(findedName)
                                        loadingImg_list[tmp] = np.dstack((loadingImg_list[tmp], image))
                                    
                                    #if tmp == 0:
                                        #print("!" + fileName + "\t" + str(tmp) + "\t" + str(len(loadingImg_list)) + "\t" +  str(image.shape) + "\t" +  str(shapetmp) + "\t" +  str(loadingImg_list[tmp].shape))
                                    #print("!" + fileName + "\t" + str(len(loadingImg_list)) + "\t" +  str(image.shape) + "\t" +  str(shapetmp))
                                else:
                                    #없으면 append
                                    loadingImg_list.append(image)
                                    #label & name 추가
                                    #loadinglable_list.append(str(len(loadingName_list))+"\t"+labelName + headingName + fileName)
                                    loadinglable_list.append(labelName)
                                    loadingName_list.append(fileName)
                                    #print("?" + fileName)
                            print()

                    #리스트에 로드된 애들 입력
                    flag = flagToType_Dic[headingName]
                    typeSize = int(len(typeName_list))
                    tmpDataSize = int(len(loadingImg_list)/typeSize)
                    cnt = 0

                    for tmp in flag:
                        tmpstr = typeName_list[tmp]
                        if labelName == labelName_list[0]:
                            image_Dic[tmpstr] = image_Dic[tmpstr] + loadingImg_list[:tmpDataSize]
                            label_Dic[tmpstr] = label_Dic[tmpstr] + loadinglable_list[:tmpDataSize]
                        else :
                            image_Dic[tmpstr] = image_Dic[tmpstr] + loadingImg_list[:tmpDataSize]
                            label_Dic[tmpstr] = label_Dic[tmpstr] + loadinglable_list[:tmpDataSize]
                         
                        target_path = os.path.join(labelDirPath, headingName)
                        target_path = os.path.join(target_path, channelName_list[-1]) # RED 이미지 경로를 저장
                        for i in range(int(len(loadingName_list)/typeSize)):
                            filePath_Dic[tmpstr].append(os.path.join(target_path, loadingName_list[i + (cnt * int(len(loadingName_list)/typeSize))]))
                            #print(tmpstr + "\t" + filePath_Dic[tmpstr][i + int(len(image_Dic[tmpstr]) - tmpDataSize)])
                            #print(tmpstr + "\t" + str(i + (cnt * int(len(loadingName_list)/typeSize))) + "\t" + str(len(filePath_Dic[tmpstr])))
                        cnt += 1
                        #print("!" + tmpstr + "\t" + str(len(image_Dic[tmpstr])) + "\t" + str(len(loadingImg_list)))
                    #print(len(image_Dic[headingName]))
    return image_Dic, label_Dic, filePath_Dic


# data 불러오기
image_Dic, label_Dic, filePath_Dic = load_data(dataset_dir, '.bmp')

# type 종류 마다 개수 확인
data_len = len(label_Dic[typeName_list[0]])
for tmp_type in typeName_list:
    if data_len != len(label_Dic[tmp_type]):
        print("DataSize ERR!!!")
        exit()

# dataset 분할
train_size = int(0.7 * data_len)
valid_size = int(0.15 * data_len)
test_size = data_len - train_size - valid_size
print("data Len : " + str(data_len))
print("(train + validation + test = total)")
print(train_size, '+', valid_size, '+', test_size, '=', train_size+valid_size+test_size)

# data shuffling
shuffled_image_Dic = {}
shuffled_label_Dic = {}
shuffled_idx_Dic  = {}

for tmp_type in typeName_list:
    if tmp_type == typeName_list[0] :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data(image_Dic[tmp_type], label_Dic[tmp_type])
    else :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data_in_defined(image_Dic[tmp_type], label_Dic[tmp_type], shuffled_idx_Dic[typeName_list[0]])

# one-hot vector로 변환
for tmp_type in typeName_list:
    shuffled_label_Dic[tmp_type] = np_utils.to_categorical(shuffled_label_Dic[tmp_type], num_classes)

x_train = {}
y_train = {}
x_valid = {}
y_valid = {}
x_test = {}
y_test = {}
shuffle_indices = {} # 디버그용
    
# dataset 구축
for tmp_type in typeName_list:
    x_train[tmp_type] = shuffled_image_Dic[tmp_type][:train_size]
    y_train[tmp_type] = shuffled_label_Dic[tmp_type][:train_size]
    x_valid[tmp_type] = shuffled_image_Dic[tmp_type][train_size:data_len-test_size]
    y_valid[tmp_type] = shuffled_label_Dic[tmp_type][train_size:data_len-test_size]
    x_test[tmp_type] = shuffled_image_Dic[tmp_type][data_len-test_size:]
    y_test[tmp_type] = shuffled_label_Dic[tmp_type][data_len-test_size:]
    shuffle_indices[tmp_type] = shuffled_idx_Dic[tmp_type][data_len-test_size:] # 디버그용
    print(tmp_type)
    print("X train : " + str(x_train[tmp_type].shape))
    print("Y train : " + str(y_train[tmp_type].shape))
    print("X test  : " + str(x_test[tmp_type].shape))
    print("X test  : " + str(y_test[tmp_type].shape))
"""
#셔플 안한거 출력 결과
for tmpType in typeName_list:
    print(tmpType)
    for dataSize in range(len(label_Dic[tmpType])):
        print("_\t" + str(filePath_Dic[tmpType][dataSize]) + "\t" + str(label_Dic[tmpType][dataSize]))  
"""

print(len(image_Dic[typeName_list[0]]))
print(image_Dic[typeName_list[0]][0].shape)
#print(image_Dic['A'])
#img = image_Dic['A'][0].reshape(325, 308, 1)
img = np.zeros((64, 304))
#print(img.shape)
#for i in range(len(filepath_list)):
#    print(filepath_list[i])

"""
for chan in range(5):
    for i in range(64):
        for j in range(304):
            img[i][j] = image_Dic['A'][5][i][j][chan]
    plt.gray()
    plt.imshow(img)
    plt.show()

print()
for tmp_type in typeName_list:
    print(tmp_type)
    for imgSize in range(len(shuffled_label_Dic[typeName_list[0]])):
        print(str(imgSize) + "\t" + str(shuffled_idx_Dic[tmp_type][imgSize]) + "\t" + str(shuffled_label_Dic[tmp_type][imgSize]) + "\t=\t" + str(label_Dic[tmp_type][shuffled_idx_Dic[tmp_type][imgSize]]))
"""
print()

0
_A
__BLUE
............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [3]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.models import Sequential, save_model

projectName_str = "1_full_dataChange_ver1"

for tmpType in typeName_list:
    path_each_str = "/" + tmpType + "/"
    model = Sequential()

    model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid', input_shape= x_train[tmpType].shape[1:]))
    model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(pool_size= (2, 2)))

    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (2, 2)))
    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (2, 2)))
    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (2, 2)))
    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size= (2, 2)))
    

    
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
    model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(2))
    model.add(Activation('softmax'))

    # configure the model
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

    # prints a summary representation of the model
    model.summary()

    # set tensorboard
    from keras.callbacks import TensorBoard
    tbCallback = TensorBoard(log_dir='./tensorboard/' + path_each_str + projectName_str)
    tbCallback.set_model(model)

    # save the model
    if not os.path.exists('./logs' + path_each_str):
        os.makedirs('./logs' + path_each_str)
    save_model(model, './logs' + path_each_str + 'my_model_' + projectName_str + '.hdf5')

    # make parameters checkpoint
    from keras.callbacks import ModelCheckpoint
    checkpoint = ModelCheckpoint(filepath="./logs/" + path_each_str + "weights.best_" + projectName_str + ".hdf5", verbose=1, 
                                   monitor='loss', save_best_only=True, mode='auto')

    # # set early stopping
    from keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # train the model
    model.fit(x_train[tmpType], y_train[tmpType],
                    epochs=50,
                    batch_size=256,
                    shuffle=False, # previously shuffled
                    validation_data=(x_valid[tmpType], y_valid[tmpType]),
                    callbacks = [tbCallback, checkpoint, early_stopping])

    '''
    learning is done at this point
    '''
    score = model.evaluate(x_test[tmpType], y_test[tmpType], verbose=0)

    print('Test loss: ', score[0])
    print('Test accuracy: ', score[1])
print("end")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 298, 16)       736       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 296, 16)       2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 148, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 148, 32)       4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 74, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 37, 32)         0         
__________

In [17]:
from keras.models import load_model

for tmpType in typeName_list:
    path_each_str = "/" + tmpType + "/"
    # restore trained model
    loaded_model = load_model('./logs' + path_each_str + 'my_model_' + projectName_str + '.hdf5')
    loaded_model.load_weights("./logs/" + path_each_str + "weights.best_" + projectName_str + ".hdf5")

    # prints a summary representation of the model
    loaded_model.summary()

    # predict test data
    predict = loaded_model.predict(x_test[tmpType])

    # show error cases
    import matplotlib.pyplot as plt
    import matplotlib.image as mpimg

    cnt = 0
    FA_cnt = 0
    FR_cnt = 0
    truesize = 0
    falsesize = 0
    for i in range(test_size):
        if np.argmax(y_test[tmpType][i]) == 0 :
            truesize += 1
        else :
            falsesize += 1
        #print(str(predict[i][0]) + '\t' + str(predict[i][1]) + '\t' + str(np.argmax(y_test[tmpType][i])))
        
        #"""
        if not (np.argmax(predict[i]) == np.argmax(y_test[tmpType][i])):
            print('file: %s' % filePath_Dic[tmpType][shuffle_indices[tmpType][i]])
            print('real: ' + str(np.argmax(y_test[tmpType][i])) + '-> predict: ' + str(np.argmax(predict[i])))
            cnt += 1
            if np.argmax(y_test[tmpType][i]) == 1:
                FA_cnt += 1
            else :
                FR_cnt += 1

            #img = mpimg.imread(filePath_Dic[tmpType][shuffle_indices[tmpType][i]])

            #plt.gray()
            #plt.imshow(img)
            #plt.show()
        #"""

    print('incorrect:', str(cnt) + '/' + str(test_size), 'False Accept:', str(FA_cnt) + '/' + str(falsesize), 'False Reject:', str(FR_cnt) + '/' + str(truesize), 'accuracy:', (test_size-cnt)/test_size)
    print('incorrect:', str(round((cnt/test_size)*100, 2)) + '%', 'False Accept:', str(round((FA_cnt/falsesize)*100, 2)) + '%', 'False Reject: ', str(round((FR_cnt/truesize)*100, 2)) + '%')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 63, 298, 16)       736       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 61, 296, 16)       2320      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 30, 148, 16)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 30, 148, 32)       4640      
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 30, 148, 32)       9248      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 15, 74, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 15, 74, 64)        18496     
__________